In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
OUT_PATH: str = '../results/manifolds/_raw'

In [4]:
DATA_PATH: str = '../data/imdb'
DATASETS: list = [
    ('train', 'sample.train'),
    ('test', 'sample.test')
]

In [5]:
DIMS: list = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]
MODELS: list = [
    ('base', 'bert-base-uncased'),
    ('textattack', 'textattack/bert-base-uncased-imdb'),
    ('fabriceyhc', 'fabriceyhc/bert-base-uncased-imdb'),
    ('wakaka', 'Wakaka/bert-finetuned-imdb')
]

In [6]:
### Load Datasets into memory

In [7]:
data_config: dict = {
    'data_label': 'text',
    'target_label': 'sentiment',
    'target_groups': {
        "negative": 0,
        "positive": 1
    }
}

In [8]:
from typing import Dict
from modules import Data

datasets: Dict[str, Data] = {
    label: Data(file_path=f"{DATA_PATH}.{name}.csv", **data_config)
    for label, name in DATASETS
}

In [9]:
for label, dataset in datasets.items():
    display(dataset.data)
    display(dataset.data['sentiment'].value_counts(normalize=True))

,text,sentiment
0,"And it falls squarely into the category of ""aw...",negative
1,This is one seriously disturbed movie. Even Th...,negative
2,"Basically this is an overlong, unfunny, action...",negative
3,Hey if you have a little over an hour to kill ...,negative
4,Did anyone read the script. This has to be som...,negative
...,...,...
1240,"First of all, Jenna Jameson is the best actres...",negative
1241,"I didnt think it was possible, but i have foun...",negative
1242,"OK, I taped this off TV and missed the very st...",negative
1243,"Okay, okay, maybe not THE greatest. I mean, Th...",positive


negative    0.503614
positive    0.496386
Name: sentiment, dtype: float64

,text,sentiment
0,This...... Movie.... Is..... Horrible!!!!!! Yo...,negative
1,At the same time John Russell was playing ranc...,positive
2,This is the best version of Gypsy that has bee...,positive
3,"It's just stories, some we wish happen to us, ...",positive
4,"This film, without doubt, is the clearest exam...",positive
...,...,...
1235,"""Cooley High"" is one of my favorite movies EVE...",positive
1236,The Comic Strip featured actors from 'The Youn...,negative
1237,I suppose you could say this film has a grain ...,negative
1238,"Having just watched Acacia, I find that I have...",negative


negative    0.520968
positive    0.479032
Name: sentiment, dtype: float64

In [10]:
### Load Encoder into Memory

In [11]:
encoder_config: dict = {
    'layers': [11]
}

In [12]:
from modules import Encoder

encoders: Dict[str, Encoder] = {
    label: Encoder({**{'model': ref}, **encoder_config})
    for label, ref in MODELS
}

In [13]:
### Compute manifolds and measure centroid distance and cluster dispersion

In [14]:
import numpy as np
import pandas as pd

In [15]:
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    # mds translates using the pairwise distances among points
    return MDS(n_components=dim, n_jobs=-1).fit_transform(data)

In [16]:
from scipy.spatial import distance

def metric_computation(groups: pd.core.groupby.GroupBy, dim: int, col: str) -> dict:

    # hold data in dict record
    record: dict = {'dim': dim}

    # centroids, intra distance per group (binary and multi class)
    for label, group in groups:

            # calculate centroid as cluster mean
            record[f'centroid_{label}'] = np.mean(
                np.stack(
                    group[col].tolist(),
                    axis=0
                ),
                axis=0
            ).tolist()

            #  calculate distance between each cluster point and centroid
            record[f'centroid_point_distances_{label}'] = distance.cdist(
                [record[f'centroid_{label}']],
                group[col].tolist()
            ).tolist()

            # calculate intra cluster distance as centroid point distance mean
            record[f'intra_distance_{label}'] = np.mean(
                record[f'centroid_point_distances_{label}'],
                axis=1
            ).item()

    # extra distance as binary centroid distance (positive|negative)
    # TODO expand to multi class
    record['extra_distance'] = distance.cdist(
        [record['centroid_positive']],
        [record['centroid_negative']]
    ).item()

    return record

In [17]:
from typing import Generator
import torch

def reduce_analyse(data: pd.DataFrame, col: str, dim: list, default_dim: int = 768) -> Generator:

    embed_col: np.ndarray = torch.stack(data[col].tolist()).numpy()

    for d in dim:

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == default_dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        yield metric_computation(
            data.groupby(dataset.target_label),
            dim=d, col='reduced_embeds'
        )

In [18]:
results: Dict[str, pd.DataFrame] = {}

In [19]:
# apply each encoder to all datasets
for enc_label, encoder in encoders.items():
    for data_label, dataset in datasets.items():

        # encode text column
        encoder.df_encode(dataset.data, col=dataset.data_label)

        # apply cluster analysis and export as dataframe
        results[f'{data_label}.{enc_label}'] = pd.DataFrame.from_records(
            list(reduce_analyse(
                dataset.data, encoder.col_name, DIMS,
                default_dim=encoder.dim)
            )
        )

In [20]:
output_cols: str = '(dim|extra_distance|intra_distance_*|centroid_point_distances_*)'

In [21]:
for label, dataset in results.items():
    results[label].filter(regex=output_cols).to_csv(f'{OUT_PATH}/{label}.csv')
    display(label, results[label].filter(regex=output_cols))

'train.base'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[5.992325561399737, 5.203575077395552, 3.5995...",5.764313,"[[6.28652306558972, 3.8467728310963367, 5.1201...",6.138071,1.259246
1,576,"[[5.996025787793756, 5.204297607458222, 3.5977...",5.765689,"[[6.306355161307022, 3.827552220961076, 5.1404...",6.139723,1.192546
2,384,"[[5.9950683167181475, 5.2000115246079375, 3.60...",5.765270,"[[6.3075686497299275, 3.8259540010354423, 5.14...",6.139276,1.196112
3,192,"[[5.993595485583582, 5.20549469507963, 3.59324...",5.763429,"[[6.297136301482993, 3.823790245628511, 5.1249...",6.137846,1.217420
4,96,"[[5.993357711694206, 5.215632068707258, 3.5688...",5.760440,"[[6.289576579087522, 3.8131488528701585, 5.132...",6.135166,1.240186
5,48,"[[5.954633792753074, 5.186424357522261, 3.5808...",5.753020,"[[6.2668352693772675, 3.7880470567407345, 5.10...",6.128055,1.311714
6,24,"[[5.990346925273335, 5.179033716094098, 3.5357...",5.739054,"[[6.309184102834411, 3.6725690582193438, 5.038...",6.116245,1.393252
7,12,"[[5.830375948686592, 5.133567925300488, 3.4097...",5.698780,"[[5.99065614517058, 3.417830585033947, 4.99785...",6.080710,1.683567
8,6,"[[5.908934528837689, 4.941959055538326, 3.3603...",5.625368,"[[6.182844579317592, 3.372902631016196, 4.9754...",6.032015,1.775144
9,3,"[[5.863928605303749, 4.771816430487086, 2.8127...",5.447228,"[[6.039240495253037, 3.1007884964652375, 4.914...",5.966240,1.331890


'test.base'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[4.832424433221461, 9.807014103979695, 3.7882...",5.798742,"[[5.229993597663103, 4.427183632815254, 5.6675...",6.084546,1.229414
1,576,"[[4.857584202387957, 9.803425183820583, 3.7994...",5.800300,"[[5.244652511383654, 4.434598608281722, 5.6629...",6.086527,1.155253
2,384,"[[4.842403996381646, 9.809672664107785, 3.7992...",5.799821,"[[5.236031814307266, 4.423988303937034, 5.6673...",6.086148,1.159657
3,192,"[[4.844491548834659, 9.798123423354937, 3.8110...",5.798409,"[[5.243993025407797, 4.44447063363301, 5.66963...",6.084642,1.175694
4,96,"[[4.829165741727322, 9.798665305208734, 3.8019...",5.795152,"[[5.238013336477926, 4.414441139088248, 5.6662...",6.081736,1.206178
5,48,"[[4.818739968202357, 9.815831903839557, 3.7931...",5.788041,"[[5.230589880993264, 4.363224339109323, 5.6586...",6.074287,1.275480
6,24,"[[4.756308977278866, 9.846593434122601, 3.7500...",5.773110,"[[5.224209636410473, 4.248390810288339, 5.6051...",6.058632,1.414432
7,12,"[[4.7055086554457235, 9.864262031975592, 3.732...",5.740305,"[[5.096355093523735, 4.042178929393498, 5.5828...",6.026544,1.602858
8,6,"[[4.6075001047767685, 9.993579902961208, 3.570...",5.683431,"[[5.131974528669825, 3.9663853588780347, 5.401...",5.984994,1.587428
9,3,"[[4.3191808667638005, 10.021234667804832, 2.21...",5.502366,"[[5.398405494344717, 3.9306227788356214, 5.530...",5.905651,1.125104


'train.textattack'

...

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[9.073984610715168, 10.79574947060581, 10.667...",9.273344,"[[3.9066403335518403, 9.816295601628925, 11.72...",7.438157,19.964325
1,576,"[[9.067112032290112, 10.79922849982439, 10.667...",9.271926,"[[3.894672502707959, 9.819489076694023, 11.730...",7.436591,19.964643
2,384,"[[9.06804295584417, 10.799514855518803, 10.668...",9.271809,"[[3.8860648233027355, 9.819043011323545, 11.73...",7.436306,19.964857
3,192,"[[9.065517495875191, 10.792411065189171, 10.67...",9.271138,"[[3.8878821729418953, 9.820797197414382, 11.73...",7.435407,19.965097
4,96,"[[9.072382455693527, 10.795707671340054, 10.67...",9.270862,"[[3.877299523115159, 9.822533962917756, 11.734...",7.434814,19.965257
5,48,"[[9.0748244277957, 10.805310910509919, 10.6749...",9.269066,"[[3.886382665896638, 9.823243231880147, 11.734...",7.432071,19.966448
6,24,"[[9.084725642630119, 10.816319532950152, 10.67...",9.263373,"[[3.8340828031805883, 9.83381354695236, 11.749...",7.423034,19.970527
7,12,"[[9.092850239297686, 10.812636111522663, 10.69...",9.251839,"[[3.770037742785303, 9.874571207139121, 11.740...",7.401691,19.977271
8,6,"[[9.163622319068022, 10.845987499295713, 10.77...",9.201321,"[[3.1515711894707263, 9.908364797917375, 11.78...",7.337691,20.009238
9,3,"[[9.083201791807651, 10.737934068500007, 10.99...",8.941231,"[[2.9536157215818837, 9.921474779636796, 11.71...",6.985183,20.238547


'train.wakaka'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[8.912249447423662, 5.991112258898744, 6.4723...",7.437458,"[[11.313891151375156, 10.330851402943722, 5.53...",8.561553,11.483087
1,576,"[[8.906167845870282, 5.978689938048081, 6.4678...",7.435450,"[[11.282162510369313, 10.304405755870572, 5.54...",8.559370,11.485576
2,384,"[[8.903244242842518, 5.97613036402344, 6.46759...",7.435200,"[[11.277535614634184, 10.299731962033366, 5.53...",8.558965,11.486265
3,192,"[[8.900178573393966, 5.9811711646898855, 6.464...",7.433424,"[[11.284723585118556, 10.294386327743073, 5.52...",8.557324,11.488861
4,96,"[[8.899865696022896, 5.962942288530846, 6.4525...",7.429967,"[[11.266481471683187, 10.292495534345214, 5.55...",8.553828,11.494947
5,48,"[[8.897541477949188, 5.969672831666577, 6.4419...",7.421431,"[[11.278751526418432, 10.300865789116669, 5.48...",8.544844,11.508065
6,24,"[[8.874721641536533, 5.9449336932781565, 6.356...",7.402399,"[[11.248067769595746, 10.281134164998585, 5.43...",8.524840,11.538541
7,12,"[[8.889472528379047, 5.858878655993993, 6.2584...",7.356873,"[[11.18556393997687, 10.232567543254962, 5.370...",8.477699,11.610363
8,6,"[[8.85760757659118, 5.585594430604996, 6.09583...",7.225882,"[[10.749021922160978, 10.123030268139741, 5.19...",8.347886,11.800688
9,3,"[[8.817791849229383, 4.519903252370375, 5.8682...",6.768356,"[[10.788607098101192, 9.730584481539557, 4.345...",7.901369,12.392536


'test.wakaka'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[6.595880875307386, 8.794097214610463, 7.2356...",7.581062,"[[6.3813950679018925, 9.885993432500822, 6.399...",8.446871,11.398275
1,576,"[[6.596735975935934, 8.791907029695935, 7.2348...",7.579149,"[[6.370918418426784, 9.868494402482803, 6.3775...",8.444711,11.401123
2,384,"[[6.591226501343642, 8.792213715012814, 7.2270...",7.578570,"[[6.37177843133338, 9.86423057698221, 6.378912...",8.444042,11.402164
3,192,"[[6.5976000460181154, 8.789446961167046, 7.228...",7.577066,"[[6.371139385280296, 9.86559214373822, 6.38024...",8.442505,11.404325
4,96,"[[6.608660585393603, 8.805609633692024, 7.2280...",7.573609,"[[6.358385726090764, 9.87006092576045, 6.36950...",8.438670,11.410169
5,48,"[[6.612873475241268, 8.796555482606898, 7.2307...",7.566044,"[[6.30200055318398, 9.826917990690303, 6.35060...",8.429627,11.422958
6,24,"[[6.539882568821202, 8.76115031952697, 7.16533...",7.547221,"[[6.27442522912879, 9.828200461754356, 6.32711...",8.409456,11.454169
7,12,"[[6.574455909808202, 8.785326728863867, 7.1427...",7.501604,"[[6.154736042406385, 9.816621617557932, 6.2084...",8.355416,11.535226
8,6,"[[6.485289562447556, 8.738885123480514, 7.1056...",7.376924,"[[5.995832477082439, 9.701045832163954, 5.9071...",8.205330,11.744548
9,3,"[[6.433110420843161, 8.986114959921165, 7.0048...",6.932552,"[[5.1057559817745695, 9.054193453346683, 5.533...",7.745594,12.324627
